In [1]:
%pip install pyautogen


[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
import autogen

config_list = autogen.config_list_from_json('OAI_CONFIG_LIST.json')

llm_config = {
    "config_list": config_list,
    "temperature": 0
    }

proxy = autogen.UserProxyAgent(name = "proxy")

coder = autogen.AssistantAgent(
    name = "Coder",
    llm_config=llm_config,
    system_message="""You are the Coder. Given a Youtube URL, write code to retrieve captions from Youtube URL. You must use `youtube-transcript-api` python library. 
        Codes you should write should populates the captions. You have to retrieve Korean captions using `YouTubeTranscriptApi.get_transcript(video_id, languages=['ko'])` method.
        You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
        Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
        If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
    """,
)

executor = autogen.UserProxyAgent(
    name="Code_executor",
    system_message="Executor. Execute the code written by the Coder and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "youtube_code", 
        "last_n_messages": 3, 
        "use_docker": False
        }
)

writer = autogen.AssistantAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="""You are the Blog Content Writer. Given a Youtube caption, write a Blog content using the captions. You must not translate the content"""
)


def state_transition(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is proxy:
        # init -> retrieve
        return coder
    elif last_speaker is coder:
        # retrieve: action 1 -> action 2
        return executor
    elif last_speaker is executor:       
        if "exitcode: 1" in messages[-1]["content"]:
            # retrieve --(execution failed)--> retrieve
            return coder
        else:
            # retrieve --(execution success)--> research
            return writer
    elif last_speaker is writer:
        # research -> end
        return None
    
groupchat = autogen.GroupChat(
    agents = [proxy, coder, executor, writer],
    messages = [],
    max_round = 20,
    speaker_selection_method=state_transition
)

chatmanager = autogen.GroupChatManager(groupchat = groupchat, llm_config = llm_config)


In [28]:
proxy.initiate_chat(chatmanager, message="https://www.youtube.com/watch?v=XjR58a6eZI0 이 유튜브 동영상을 이용하여 블로그 콘텐츠를 만들어줘")

proxy (to chat_manager):

https://www.youtube.com/watch?v=XjR58a6eZI0 이 유튜브 동영상을 이용하여 블로그 콘텐츠를 만들어줘

--------------------------------------------------------------------------------

Next speaker: Coder

Coder (to chat_manager):

먼저, `youtube-transcript-api` 라이브러리를 사용하여 주어진 유튜브 동영상의 한국어 자막을 가져오는 파이썬 코드를 작성하겠습니다. 

다음은 해당 작업을 수행하는 전체 파이썬 코드입니다:

```python
import sys
from youtube_transcript_api import YouTubeTranscriptApi

def get_korean_captions(video_url):
    try:
        # Extract the video ID from the URL
        video_id = video_url.split('v=')[1]
        # Retrieve the Korean captions
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko'])
        return transcript
    except Exception as e:
        return str(e)

if __name__ == "__main__":
    video_url = "https://www.youtube.com/watch?v=XjR58a6eZI0"
    captions = get_korean_captions(video_url)
    if isinstance(captions, list):
        for caption in captions:
            print(f"{caption['start']}:

ChatResult(chat_id=None, chat_history=[{'content': 'https://www.youtube.com/watch?v=XjR58a6eZI0 이 유튜브 동영상을 이용하여 블로그 콘텐츠를 만들어줘', 'role': 'assistant'}, {'content': '먼저, `youtube-transcript-api` 라이브러리를 사용하여 주어진 유튜브 동영상의 한국어 자막을 가져오는 파이썬 코드를 작성하겠습니다. \n\n다음은 해당 작업을 수행하는 전체 파이썬 코드입니다:\n\n```python\nimport sys\nfrom youtube_transcript_api import YouTubeTranscriptApi\n\ndef get_korean_captions(video_url):\n    try:\n        # Extract the video ID from the URL\n        video_id = video_url.split(\'v=\')[1]\n        # Retrieve the Korean captions\n        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[\'ko\'])\n        return transcript\n    except Exception as e:\n        return str(e)\n\nif __name__ == "__main__":\n    video_url = "https://www.youtube.com/watch?v=XjR58a6eZI0"\n    captions = get_korean_captions(video_url)\n    if isinstance(captions, list):\n        for caption in captions:\n            print(f"{caption[\'start\']}: {caption[\'text\']}")\n    else:\n   